In [1]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Sumanandana\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
fakedet_news = pd.read_csv('train.csv')

In [3]:
fakedet_news.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
fakedet_news.shape

(20800, 5)

In [5]:
fakedet_news.isna().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [6]:
fakedet_news = fakedet_news.fillna(' ')

In [7]:
fakedet_news.isna().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [8]:
fakedet_news['content'] = fakedet_news['author']+' '+fakedet_news['title']

In [9]:
fakedet_news

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...
...,...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0,Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0,"Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma..."
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0,Michael J. de la Merced and Rachel Abrams Macy...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1,"Alex Ansary NATO, Russia To Hold Parallel Exer..."


In [10]:
fakedet_news['content'][20796]

'Benjamin Hoffman N.F.L. Playoffs: Schedule, Matchups and Odds - The New York Times'

In [11]:
ps = PorterStemmer()
def stemming(content):
    stemmed_content = re.sub('[^a-zA_Z]'," ",content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [ps.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = " ".join(stemmed_content)
    return stemmed_content

In [12]:
fakedet_news['content'] = fakedet_news['content'].apply(stemming)

In [13]:
fakedet_news['content']

0        arrel ucu ous em aid e idn ven ee omey etter n...
1           aniel lynn illari linton ig oman ampu reitbart
2                  onsortiumnew com hy ruth ight et ou ire
3        essica urkiss ivilian ill n ingl airstrik ave ...
4        oward ortnoy ranian woman jail fiction unpubli...
                               ...                        
20795       erom udson apper rump oster hild hite upremaci
20796    enjamin offman layoff chedul atchup dd ew ork ime
20797    ichael de la erc achel abram aci aid eceiv ake...
20798         alex ansari ussia old arallel xercis n alkan
20799                             avid wanson hat eep aliv
Name: content, Length: 20800, dtype: object

In [14]:
X = fakedet_news['content'].values
Y = fakedet_news['label'].values

In [15]:
print(X)

['arrel ucu ous em aid e idn ven ee omey etter ntil ason haffetz weet'
 'aniel lynn illari linton ig oman ampu reitbart'
 'onsortiumnew com hy ruth ight et ou ire' ...
 'ichael de la erc achel abram aci aid eceiv akeov approach udson ay ew ork ime'
 'alex ansari ussia old arallel xercis n alkan' 'avid wanson hat eep aliv']


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [17]:
vector = TfidfVectorizer()
vector.fit(X)
X = vector.transform(X)

In [18]:
print(X)

  (0, 16091)	0.2750955991151135
  (0, 15873)	0.2571976706442014
  (0, 14997)	0.330935212153767
  (0, 11824)	0.20185650488848295
  (0, 10862)	0.2286181575281217
  (0, 10230)	0.3022167726538608
  (0, 7389)	0.2783581199034822
  (0, 6508)	0.3468000816494851
  (0, 5853)	0.25342341361447035
  (0, 4969)	0.24916622073216285
  (0, 4518)	0.2118104090933208
  (0, 2268)	0.21894421086060042
  (0, 2061)	0.31730681321499543
  (0, 583)	0.21624468658492996
  (1, 12991)	0.19892561984000542
  (1, 10808)	0.3901807845495569
  (1, 9201)	0.45773341435725207
  (1, 8957)	0.24717748739783135
  (1, 7644)	0.24590623033830125
  (1, 7470)	0.3500921438471845
  (1, 1388)	0.3375600812230468
  (1, 1192)	0.4904995132577478
  (2, 13736)	0.38505168207348944
  (2, 11720)	0.32106313955963073
  (2, 11108)	0.45862337972873657
  :	:
  (20797, 7719)	0.0868281349953222
  (20797, 7282)	0.18580559202052796
  (20797, 5954)	0.08188165916518761
  (20797, 5406)	0.31422059592151225
  (20797, 4358)	0.2940127910277199
  (20797, 3828)	0.2

In [19]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=1)

In [20]:
X_train.shape

(16640, 16631)

In [21]:
X_test.shape

(4160, 16631)

In [22]:
model = LogisticRegression()
model.fit(X_train,Y_train)

LogisticRegression()

In [23]:
train_Y_pred = model.predict(X_train)
print("train accuracy :",accuracy_score(train_Y_pred,Y_train))

train accuracy : 0.9847355769230769


In [24]:
test_Y_pred = model.predict(X_test)
print("train accuracy :",accuracy_score(test_Y_pred,Y_test))

train accuracy : 0.9730769230769231


In [25]:
input_data = X_test[20]
prediction = model.predict(input_data)
if prediction[0] == 1:
    print('Fake news')
else:
    print('Real news')

Fake news


In [26]:
fakedet_news['content'][20]

'ew ope ude aul yan ust merg rom ayahuasca ent ith ision f ew epublican arti'